In [24]:
# Question 1
# (a)
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, naive_bayes, metrics
from numpy import linalg as LA
from sklearn.metrics import confusion_matrix
import re
import string
import random
from nltk.stem.wordnet import WordNetLemmatizer

yelp = pd.read_table("/Desktop/cs5785/hw3/sentiment_labelled_sentences/yelp_labelled.txt", header = None, sep = "\t+", engine = 'python').values
amazon_cells = pd.read_table("/Desktop/cs5785/hw3/sentiment_labelled_sentences/amazon_cells_labelled.txt", header = None, sep = "\t+", engine = 'python').values
imdb = pd.read_table("/Desktop/cs5785/hw3/sentiment_labelled_sentences/imdb_labelled.txt", header = None, sep = "\t+", engine = 'python').values
print(yelp.shape)
print(amazon_cells.shape)
print(imdb.shape)


(1000, 2)
(1000, 2)
(1000, 2)


In [6]:
print "Number of 1's in yelp: ", np.sum(yelp[:, 1] != 0)
print "Number of 0's in yelp: ", np.sum(yelp[:, 1] == 0)
print "Number of 1's in amazon cells: ", np.sum(amazon_cells[:, 1] != 0)
print "Number of 0's in amazon cells: ", np.sum(amazon_cells[:, 1] == 0)
print "Number of 1's in imdb: ", np.sum(imdb[:, 1] != 0)
print "Number of 0's in imdb: ", np.sum(imdb[:, 1] == 0)


Number of 1's in yelp:  500
Number of 0's in yelp:  500
Number of 1's in amazon cells:  500
Number of 0's in amazon cells:  500
Number of 1's in imdb:  500
Number of 0's in imdb:  500


In [12]:
# (b)-(g)

yelp_train = np.append(yelp[np.where(yelp[:, 1] == 0)][:400], yelp[np.where(yelp[:, 1] != 0)][:400], axis = 0)
print yelp_train.shape
yelp_test = np.append(yelp[np.where(yelp[:, 1] == 0)][400:], yelp[np.where(yelp[:, 1] != 0)][400:], axis = 0)
print yelp_test.shape

amazon_cells_train = np.append(amazon_cells[np.where(amazon_cells[:, 1] == 0)][:400], amazon_cells[np.where(amazon_cells[:, 1] != 0)][:400], axis = 0)
print amazon_cells_train.shape
amazon_cells_test = np.append(amazon_cells[np.where(amazon_cells[:, 1] == 0)][400:], amazon_cells[np.where(amazon_cells[:, 1] != 0)][400:], axis = 0)
print amazon_cells_test.shape

imdb_train = np.append(imdb[np.where(imdb[:, 1] == 0)][:400], imdb[np.where(imdb[:, 1] != 0)][:400], axis = 0)
print imdb_train.shape
imdb_test = np.append(imdb[np.where(imdb[:, 1] == 0)][400:], imdb[np.where(imdb[:, 1] != 0)][400:], axis = 0)
print imdb_test.shape

train = np.append(np.append(yelp_train, amazon_cells_train, axis = 0), imdb_train, axis = 0)
test = np.append(np.append(yelp_test, amazon_cells_test, axis = 0), imdb_test, axis = 0)
print train.shape, test.shape

(800, 2)
(200, 2)
(800, 2)
(200, 2)
(800, 2)
(200, 2)
(2400, 2) (600, 2)


In [13]:
class BagOfWordsVectorizer(object):
    def __init__(self):
        self.func_lemmatize = WordNetLemmatizer().lemmatize

        self.STRIP_CHARS = string.punctuation + ' '
        self.PUNCTUATIONS = set(string.punctuation)

#         self.STOP_WORDS = set(stopwords.words('english'))
        self.STOP_WORDS = set([])


        self.APOSTROPHE_DICT = {"don't": "donot", "doesn't": "donot", "didn't": "donot", 
                                "isn't": "isnot", "aren't": "isnot", 
                                "i'm": "i", "you're": "you", "they're": "they", "he's": "he", "she's": "she", 
                                "it's": "it", "i've": "i", "you've": "you",
                                "ha": "has"}
        self._vocabulary = None
        self._lookup_voc = None
        
    def _strip_and_lemmatize(self, word, chars):
#         word = str(self.func_lemmatize(word.strip(chars)))
        word = str(self.func_lemmatize(word))
        
        if word in self.APOSTROPHE_DICT:
            word = self.APOSTROPHE_DICT[word]
            
#         return word
        return word.translate(None, self.STRIP_CHARS)
    
    def _tokenize_and_clean(self, sentence):
#         _ = [self._strip_and_lemmatize(s, self.STRIP_CHARS) 
#              for s in sentence.lower().split(' ') if (s not in self.STOP_WORDS and len(s) > 0)]
        _ = [self._strip_and_lemmatize(s, self.STRIP_CHARS) 
             for s in re.split(r'[^a-z\']', sentence.lower()) if (s not in self.STOP_WORDS and len(s) > 0)]
    
        return filter(lambda s: len(s) > 0 and 
                               s not in self.STOP_WORDS and 
                               s not in self.PUNCTUATIONS, _)
    def tokenize(self, X):
        return map(self._tokenize_and_clean, X)
    
    
    def _l1(self, x):
        _l = len(x.nonzero()[0])
        if _l > 0:
            return x / float(_l)
        else:
            return np.zeros((len(x),))
        
    def _l2(self, x):
        _l = len(x.nonzero()[0])
        if _l > 0:
            return x / float(np.sqrt(np.dot(x, x.T)))
        else:
            return np.zeros((len(x),))

    def l1_normalize(self, X):
        return np.apply_along_axis(lambda x: self._l1(x), 1, X)

    def l2_normalize(self, X):
        return np.apply_along_axis(lambda x: self._l2(x), 1, X)
    
    
    def _bag_of_words(self, words):
        bag = np.zeros((self._voc_len,), dtype=np.int)
        for word in words:
            try:
                bag[self._vocabulary[word]] += 1
            except KeyError:
                continue
        return bag
    
        
    def fit(self, X):
        self.fit_transform(X)
        return self
    
    def fit_transform(self, X, normalize=None):
        # 1. build vocabulary
        tokens = self.tokenize(X)
        
        _ = set(np.concatenate(tokens))
        
        self._lookup_voc = {}
        self._vocabulary = {}
        for i, x in enumerate(_):
            self._lookup_voc[i] = x
            self._vocabulary[x] = i
        
        self._voc_len = len(_)
        
        # 2. generate X
        _ = np.array(map(self._bag_of_words, tokens))
        if normalize is None:
            return _
        elif normalize == 'l2':
            return self.l2_normalize(_)
        else:
            return self.l1_normalize(_)
            
    def transform(self, X, normalize=None):
        tokens = self.tokenize(X)
#         print tokens
        if self._vocabulary is not None:
            _ = np.array(map(self._bag_of_words, tokens)) 
            
            if normalize is None:
                return _
            elif normalize == 'l2':
                return self.l2_normalize(_)
            else:
                return self.l1_normalize(_)
        else:
            return None

In [14]:
class NGramVectorizer(BagOfWordsVectorizer):
    def __init__(self, N):
        super(NGramVectorizer, self).__init__()
        self.N = N
        
    def _gen_ngram(self, tokens):
        r = []
        for i in range(0, len(tokens) - self.N + 1):
            r.append(" ".join(tokens[i:i+self.N]))
        return r
        
    def tokenize(self, X):
        _ = super(NGramVectorizer, self).tokenize(X)
        return map(self._gen_ngram, _)

In [21]:
train_labels = train[:, 1].astype(int)
test_labels = test[:, 1].astype(int)
print train_labels.shape
print test_labels.shape

(2400,)
(600,)


In [58]:
# Bag-of-Words
# Logistic regression
print "Bag-of-Word Model with Logistic Regression"

normalize = 'l2'

bowv = BagOfWordsVectorizer()
train_bag = bowv.fit_transform(train[:,0], normalize)
test_bag = bowv.transform(test[:,0], normalize)
print(train_bag.shape)
print(test_bag.shape)
print(train_bag[500])
print(train_bag[1000])

cls_lr = linear_model.LogisticRegression()
cls_lr.fit(train_bag, train_labels)
score_lr = cls_lr.fit(train_bag, train_labels).score(test_bag, test_labels)
pred_lr = cls_lr.predict(test_bag)

print "\nClassification accuracy: ", score_lr
print metrics.confusion_matrix(test_labels, pred_lr)
print '-----'

coef = cls.coef_[0]
idx = np.argsort(coef)
print 'Positive words: '
for i in idx[::-1][:20]:
    print '%s; ' % bowv._lookup_voc[i],
print '\n-----'
print 'Negative words: '
for i in idx[:20]:
    print '%s; ' % bowv._lookup_voc[i],


Bag-of-Word Model with Logistic Regression
(2400, 4179)
(600, 4179)
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]

Classification accuracy:  0.785
[[243  57]
 [ 72 228]]
-----
Positive words: 
great;  good;  love;  excellent;  best;  nice;  delicious;  amazing;  loved;  fantastic;  awesome;  well;  funny;  and;  liked;  recommend;  definitely;  friendly;  price;  happy;  
-----
Negative words: 
not;  bad;  donot;  poor;  worst;  terrible;  awful;  no;  disappointing;  never;  horrible;  there;  minute;  disappointed;  waste;  then;  at;  disappointment;  slow;  hour; 


In [59]:
# Bag-of-Words
# Naive Bayes
print "Bag-of-Word Model with Naive Bayes"

normalize = 'l2'
bowv = BagOfWordsVectorizer()
train_bag = bowv.fit_transform(train[:,0], normalize)
test_bag = bowv.transform(test[:,0], normalize)
print(train_bag.shape)
print(test_bag.shape)
print(train_bag[500])
print(train_bag[1000])

cls_nb = naive_bayes.BernoulliNB()
cls_nb.fit(train_bag, train_labels)
score_nb = cls_nb.fit(train_bag, train_labels).score(test_bag, test_labels)
pred_nb = cls_nb.predict(test_bag)

print "\nClassification accuracy: ", score_nb
print metrics.confusion_matrix(test_labels, pred_nb)
print '-----'

coef = cls.coef_[0]
idx = np.argsort(coef)
print 'Positive words: '
for i in idx[::-1][:20]:
    print '%s; ' % bowv._lookup_voc[i],
print '\n-----'
print 'Negative words: '
for i in idx[:20]:
    print '%s; ' % bowv._lookup_voc[i],


Bag-of-Word Model with Naive Bayes
(2400, 4179)
(600, 4179)
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]

Classification accuracy:  0.81
[[256  44]
 [ 70 230]]
-----
Positive words: 
great;  good;  love;  excellent;  best;  nice;  delicious;  amazing;  loved;  fantastic;  awesome;  well;  funny;  and;  liked;  recommend;  definitely;  friendly;  price;  happy;  
-----
Negative words: 
not;  bad;  donot;  poor;  worst;  terrible;  awful;  no;  disappointing;  never;  horrible;  there;  minute;  disappointed;  waste;  then;  at;  disappointment;  slow;  hour; 


In [60]:
# N-gram
# Logistic regression
print "N-Gram Model with Logistic Regression"

normalize = 'l2'

bowv = NGramVectorizer(2)
train_ngram = bowv.fit_transform(train[:,0], normalize)
test_ngram = bowv.transform(test[:,0], normalize)
print(train_ngram.shape)
print(test_ngram.shape)
print(train_ngram[500])
print(train_ngram[1000])

cls_lr = linear_model.LogisticRegression()
cls_lr.fit(train_ngram, train_labels)
score_lr = cls_lr.fit(train_ngram, train_labels).score(test_ngram, test_labels)
pred_lr = cls_lr.predict(test_ngram)

print "\nClassification accuracy: ", score_lr
print metrics.confusion_matrix(test_labels, pred_lr)
print '-----'

coef = cls.coef_[0]
idx = np.argsort(coef)
print 'Positive words: '
for i in idx[::-1][:20]:
    print '%s; ' % bowv._lookup_voc[i],
print '\n-----'
print 'Negative words: '
for i in idx[:20]:
    print '%s; ' % bowv._lookup_voc[i],

N-Gram Model with Logistic Regression
(2400, 16417)
(600, 16417)
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]

Classification accuracy:  0.716666666667
[[233  67]
 [103 197]]
-----
Positive words: 
unbearable after;  meat and;  cinema imaginable;  back many;  i now;  coastal retreat;  how hollywood;  walked in;  a stunning;  problem wit;  yell when;  builder were;  about useless;  created a;  with hatred;  player in;  that ballet;  such a;  also came;  cinematography isnot;  
-----
Negative words: 
nyc bagel;  of ebay;  wa pink;  lunch and;  play but;  road house;  so mature;  the selection;  it can;  people like;  fx are;  our side;  the wrong;  realize that;  behind this;  jabra behing;  out the;  working in;  authentic or;  upstairs and; 


In [61]:
# N-gram
# Naive Bayes
print "N-Gram Model with Naive Bayes"

normalize = 'l2'

bowv = NGramVectorizer(2)
train_ngram = bowv.fit_transform(train[:,0], normalize)
test_ngram = bowv.transform(test[:,0], normalize)
print(train_ngram.shape)
print(test_ngram.shape)
print(train_ngram[500])
print(train_ngram[1000])

cls_nb = naive_bayes.BernoulliNB()
cls_nb.fit(train_ngram, train_labels)
score_nb = cls_nb.fit(train_ngram, train_labels).score(test_ngram, test_labels)
pred_nb = cls_nb.predict(test_ngram)

print "\nClassification accuracy: ", score_nb
print metrics.confusion_matrix(test_labels, pred_nb)
print '-----'

coef = cls.coef_[0]
idx = np.argsort(coef)
print 'Positive words: '
for i in idx[::-1][:20]:
    print '%s; ' % bowv._lookup_voc[i],
print '\n-----'
print 'Negative words: '
for i in idx[:20]:
    print '%s; ' % bowv._lookup_voc[i],

N-Gram Model with Naive Bayes
(2400, 16417)
(600, 16417)
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]

Classification accuracy:  0.74
[[202  98]
 [ 58 242]]
-----
Positive words: 
unbearable after;  meat and;  cinema imaginable;  back many;  i now;  coastal retreat;  how hollywood;  walked in;  a stunning;  problem wit;  yell when;  builder were;  about useless;  created a;  with hatred;  player in;  that ballet;  such a;  also came;  cinematography isnot;  
-----
Negative words: 
nyc bagel;  of ebay;  wa pink;  lunch and;  play but;  road house;  so mature;  the selection;  it can;  people like;  fx are;  our side;  the wrong;  realize that;  behind this;  jabra behing;  out the;  working in;  authentic or;  upstairs and; 


In [45]:
# (h)

class PCA(object):
    def fit_transform(self, X, q):
        self.u = np.mean(X, axis=0)
        self._X = X - self.u
        self.V = LA.svd(self._X)[-1]
        return self._X.dot(self.V[:q, :].T)
    
    def transform(self, X, q):
        return (X - self.u).dot(self.V[:q, :].T)

In [50]:
# Bag-of-Word Model with Logistic Regression after PCA

print "Bag-of-Word Model with Logistic Regression after PCA"

normalize = 'l2'
    
for q in [10, 50, 100]:
    print '\n-----\nRank: ', q
    
    bowv = BagOfWordsVectorizer()
    pca = PCA()
    train_bag = pca.fit_transform(bowv.fit_transform(train[:,0], normalize), q)
    test_bag = pca.transform(bowv.transform(test[:,0], normalize), q)

    cls_lr = linear_model.LogisticRegression()
    cls_lr.fit(train_bag, train_labels)
    score_lr = cls_lr.fit(train_bag, train_labels).score(test_bag, test_labels)
    pred_lr = cls_lr.predict(test_bag)

    print "\nClassification accuracy: ", score_lr
    print metrics.confusion_matrix(test_labels, pred_lr)
    print '-----'

    coef = cls.coef_[0]
    idx = np.argsort(coef)
    print 'Positive words: '
    for i in idx[::-1][:20]:
        print '%s; ' % bowv._lookup_voc[i],
    print '\n-----'
    print 'Negative words: '
    for i in idx[:20]:
        print '%s; ' % bowv._lookup_voc[i],

Bag-of-Word Model with Logistic Regression after PCA

-----
Rank:  10

Classification accuracy:  0.583333333333
[[197 103]
 [147 153]]
-----
Positive words: 
great;  good;  love;  excellent;  best;  nice;  delicious;  amazing;  loved;  fantastic;  awesome;  well;  funny;  and;  liked;  recommend;  definitely;  friendly;  price;  happy;  
-----
Negative words: 
not;  bad;  donot;  poor;  worst;  terrible;  awful;  no;  disappointing;  never;  horrible;  there;  minute;  disappointed;  waste;  then;  at;  disappointment;  slow;  hour;  
-----
Rank:  50

Classification accuracy:  0.671666666667
[[210  90]
 [107 193]]
-----
Positive words: 
great;  good;  love;  excellent;  best;  nice;  delicious;  amazing;  loved;  fantastic;  awesome;  well;  funny;  and;  liked;  recommend;  definitely;  friendly;  price;  happy;  
-----
Negative words: 
not;  bad;  donot;  poor;  worst;  terrible;  awful;  no;  disappointing;  never;  horrible;  there;  minute;  disappointed;  waste;  then;  at;  disa

In [62]:
# Bag-of-Word Model with Naive Bayes after PCA

print "Bag-of-Word Model with Naive Bayes after PCA"

normalize = 'l2'
    
for q in [10, 50, 100]:
    print '\n-----\nRank: ', q
    
    bowv = BagOfWordsVectorizer()
    pca = PCA()
    train_bag = pca.fit_transform(bowv.fit_transform(train[:,0], normalize), q)
    test_bag = pca.transform(bowv.transform(test[:,0], normalize), q)

    cls_nb = naive_bayes.BernoulliNB()
    cls_nb.fit(train_bag, train_labels)
    score_nb = cls_nb.fit(train_bag, train_labels).score(test_bag, test_labels)
    pred_nb = cls_nb.predict(test_bag)

    print "\nClassification accuracy: ", score_nb
    print metrics.confusion_matrix(test_labels, pred_nb)
    print '-----'

    coef = cls.coef_[0]
    idx = np.argsort(coef)
    print 'Positive words: '
    for i in idx[::-1][:20]:
        print '%s; ' % bowv._lookup_voc[i],
    print '\n-----'
    print 'Negative words: '
    for i in idx[:20]:
        print '%s; ' % bowv._lookup_voc[i],

Bag-of-Word Model with Naive Bayes after PCA

-----
Rank:  10

Classification accuracy:  0.555
[[170 130]
 [137 163]]
-----
Positive words: 
great;  good;  love;  excellent;  best;  nice;  delicious;  amazing;  loved;  fantastic;  awesome;  well;  funny;  and;  liked;  recommend;  definitely;  friendly;  price;  happy;  
-----
Negative words: 
not;  bad;  donot;  poor;  worst;  terrible;  awful;  no;  disappointing;  never;  horrible;  there;  minute;  disappointed;  waste;  then;  at;  disappointment;  slow;  hour;  
-----
Rank:  50

Classification accuracy:  0.601666666667
[[181 119]
 [120 180]]
-----
Positive words: 
great;  good;  love;  excellent;  best;  nice;  delicious;  amazing;  loved;  fantastic;  awesome;  well;  funny;  and;  liked;  recommend;  definitely;  friendly;  price;  happy;  
-----
Negative words: 
not;  bad;  donot;  poor;  worst;  terrible;  awful;  no;  disappointing;  never;  horrible;  there;  minute;  disappointed;  waste;  then;  at;  disappointment;  slow

In [52]:
# N-Gram Model with Logistic Regression after PCA

print "N-Gram Model with Logistic Regression after PCA"

normalize = 'l2'
    
for q in [10, 50, 100]:
    print '\n-----\nRank: ', q
    
    bowv = NGramVectorizer(2)
    pca = PCA()
    train_ngram = pca.fit_transform(bowv.fit_transform(train[:,0], normalize), q)
    test_ngram = pca.transform(bowv.transform(test[:,0], normalize), q)

    cls_lr = linear_model.LogisticRegression()
    cls_lr.fit(train_ngram, train_labels)
    score_lr = cls_lr.fit(train_ngram, train_labels).score(test_ngram, test_labels)
    pred_lr = cls_lr.predict(test_ngram)

    print "\nClassification accuracy: ", score_lr
    print metrics.confusion_matrix(test_labels, pred_lr)
    print '-----'

    coef = cls.coef_[0]
    idx = np.argsort(coef)
    print 'Positive words: '
    for i in idx[::-1][:20]:
        print '%s; ' % bowv._lookup_voc[i],
    print '\n-----'
    print 'Negative words: '
    for i in idx[:20]:
        print '%s; ' % bowv._lookup_voc[i],

N-Gram Model with Logistic Regression after PCA

-----
Rank:  10

Classification accuracy:  0.543333333333
[[241  59]
 [215  85]]
-----
Positive words: 
unbearable after;  meat and;  cinema imaginable;  back many;  i now;  coastal retreat;  how hollywood;  walked in;  a stunning;  problem wit;  yell when;  builder were;  about useless;  created a;  with hatred;  player in;  that ballet;  such a;  also came;  cinematography isnot;  
-----
Negative words: 
nyc bagel;  of ebay;  wa pink;  lunch and;  play but;  road house;  so mature;  the selection;  it can;  people like;  fx are;  our side;  the wrong;  realize that;  behind this;  jabra behing;  out the;  working in;  authentic or;  upstairs and;  
-----
Rank:  50

Classification accuracy:  0.598333333333
[[215  85]
 [156 144]]
-----
Positive words: 
unbearable after;  meat and;  cinema imaginable;  back many;  i now;  coastal retreat;  how hollywood;  walked in;  a stunning;  problem wit;  yell when;  builder were;  about useless;  cr

In [63]:
# N-Gram Model with Naive Bayes after PCA

print "N-Gram Model with Naive Bayes after PCA"

normalize = 'l2'
    
for q in [10, 50, 100]:
    print '\n-----\nRank: ', q
    
    bowv = NGramVectorizer(2)
    pca = PCA()
    train_ngram = pca.fit_transform(bowv.fit_transform(train[:,0], normalize), q)
    test_ngram = pca.transform(bowv.transform(test[:,0], normalize), q)

    cls_nb = naive_bayes.BernoulliNB()
    cls_nb.fit(train_ngram, train_labels)
    score_nb = cls_nb.fit(train_ngram, train_labels).score(test_ngram, test_labels)
    pred_nb = cls_nb.predict(test_ngram)

    print "\nClassification accuracy: ", score_nb
    print metrics.confusion_matrix(test_labels, pred_nb)
    print '-----'

    coef = cls.coef_[0]
    idx = np.argsort(coef)
    print 'Positive words: '
    for i in idx[::-1][:20]:
        print '%s; ' % bowv._lookup_voc[i],
    print '\n-----'
    print 'Negative words: '
    for i in idx[:20]:
        print '%s; ' % bowv._lookup_voc[i],

N-Gram Model with Naive Bayes after PCA

-----
Rank:  10

Classification accuracy:  0.558333333333
[[182 118]
 [147 153]]
-----
Positive words: 
unbearable after;  meat and;  cinema imaginable;  back many;  i now;  coastal retreat;  how hollywood;  walked in;  a stunning;  problem wit;  yell when;  builder were;  about useless;  created a;  with hatred;  player in;  that ballet;  such a;  also came;  cinematography isnot;  
-----
Negative words: 
nyc bagel;  of ebay;  wa pink;  lunch and;  play but;  road house;  so mature;  the selection;  it can;  people like;  fx are;  our side;  the wrong;  realize that;  behind this;  jabra behing;  out the;  working in;  authentic or;  upstairs and;  
-----
Rank:  50

Classification accuracy:  0.598333333333
[[179 121]
 [120 180]]
-----
Positive words: 
unbearable after;  meat and;  cinema imaginable;  back many;  i now;  coastal retreat;  how hollywood;  walked in;  a stunning;  problem wit;  yell when;  builder were;  about useless;  created a;